In [1]:
import torch
import os

# Pour le debug, je met la variable d'environnement CUDA_LAUNCH_BLOCKING=1
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Utilisation du périphérique : {device}")
torch.cuda.get_device_name()


Utilisation du périphérique : cuda


'NVIDIA GeForce RTX 3080 Ti'

# Chargement du dataset

In [2]:
from datasets import load_dataset

nom_dataset = "bigcode/the-stack-smol-xl"
nom_model = "Salesforce/codegen-350M-mono"
# nom_model = "distilgpt2"

# dataset = load_dataset("bigcode/the-stack-smol-xl", data_dir="data/dockerfile")
dataset = load_dataset(nom_dataset, data_dir="data/python", split="train[:20]")

dataset

README.md: 0.00B [00:00, ?B/s]

data.json:   0%|          | 0.00/96.4M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['hexsha', 'size', 'ext', 'lang', 'max_stars_repo_path', 'max_stars_repo_name', 'max_stars_repo_head_hexsha', 'max_stars_repo_licenses', 'max_stars_count', 'max_stars_repo_stars_event_min_datetime', 'max_stars_repo_stars_event_max_datetime', 'max_issues_repo_path', 'max_issues_repo_name', 'max_issues_repo_head_hexsha', 'max_issues_repo_licenses', 'max_issues_count', 'max_issues_repo_issues_event_min_datetime', 'max_issues_repo_issues_event_max_datetime', 'max_forks_repo_path', 'max_forks_repo_name', 'max_forks_repo_head_hexsha', 'max_forks_repo_licenses', 'max_forks_count', 'max_forks_repo_forks_event_min_datetime', 'max_forks_repo_forks_event_max_datetime', 'content', 'avg_line_length', 'max_line_length', 'alphanum_fraction'],
    num_rows: 20
})

# Chargement du model


In [3]:
from opco_4_utils import init_model, do_predict

#tokenizer, model = init_model("Salesforce/codegen-350M-mono")
tokenizer, model = init_model(nom_model)




tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Tokenizer par defaut


In [4]:
# On ne conserve que le code car rien d'autre interessant dans le dataset

dataset = dataset.map(
    lambda ex: {
        'code': ex['content']
    },
    remove_columns=dataset.column_names
)

dataset

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['code'],
    num_rows: 20
})

In [14]:
print(dataset[3]['code'])

import time
import logging
import httplib as http_client
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
from requests.packages.urllib3.util.retry import Retry
import ssl
import base64
try:
    from cStringIO import StringIO
except BaseException:
    from StringIO import StringIO
import zipfile


class MyAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = PoolManager(num_pools=connections,
                                       maxsize=maxsize,
                                       block=block,
                                       ssl_version=getattr(ssl, 'PROTOCOL_TLSv1_2', ssl.PROTOCOL_TLSv1))

# Retry logic if the API fails to responde


def requests_retry_session(
        retries=5,
        backoff_factor=0.5,
        status_forcelist=(500, 502, 504, 495, 496, 525, 526),
        session=None,
):
    session = session or requests.Session()
    re

Le dataset est préparer

il faut maintenant 'tokeniser' les lignes

In [6]:
TOKENIZER_MAX_LENGTH = 2048 # Longuer maximum de séquence, visiblement le model accepte pas plus de 2048 tokens
tokenised_dataset = dataset.map(
    lambda ex: tokenizer(
        ex['code'], # Colonne a truncate
        truncation=True, # On truncate donc les input
        max_length=TOKENIZER_MAX_LENGTH, # TOKENIZER_MAX_LENGTH (2048) max
        return_tensors="pt", # TODO: A approfondir
        padding="max_length" # séquence de token < TOKENIZER_MAX_LENGHTH -> on fill la séquence jusqu'a la longueur max, voir l'avantage/raison
    )
)

tokenised_dataset

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['code', 'input_ids', 'attention_mask'],
    num_rows: 20
})

In [10]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Définir le répertoire de sortie pour le modèle fine-tuné
output_model_file = os.path.join("export_model", "./codegen-350m-mono-finetuned-python")

training_args = TrainingArguments(
    output_dir=output_model_file,
    overwrite_output_dir=True,
    num_train_epochs=1,  # Un seul epoch peut déjà donner de bons résultats
    per_device_train_batch_size=1,  # Taille du batch, a faire varier en fonction de la ram dispo
    gradient_accumulation_steps=2, # Simule un batch size plus grand (4*2=8)
    save_steps=500,
    save_total_limit=2,
    learning_rate=2e-5,
 #   fp16=True,  # Utiliser le mixed precision pour accélérer l'entraînement (nécessite un GPU compatible)
    logging_steps=500,
#    report_to="tensorboard", # Optionnel: pour visualiser les logs
    use_cpu=True,
    gradient_checkpointing=False
)

training_args = TrainingArguments(
    output_dir="./results_test_final",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    use_cpu=True, # On reste sur CPU pour le test
)


In [11]:
tonclean = tokenised_dataset.remove_columns(["code"])
tonclean

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 20
})

In [13]:
# Le Data Collator prépare les batches pour le language modeling
# TODO: mlm=False car c'est du Causal LM (style GPT), pas du Masked LM (style BERT) à voir la nuance
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised_dataset,
    data_collator=data_collator
    # ,
    # tokenizer=tokenizer
)

# Lancer le fine-tuning 🚀
trainer.train()

# Sauvegarder le modèle final
# trainer.save_model(output_model_file)
# print(f"Modèle sauvegardé dans {output_model_file}")

RuntimeError: expand(torch.FloatTensor{[2, 2, 1, 1, 2048]}, size=[2, 1, 1, 2048]): the number of sizes provided (4) must be greater or equal to the number of dimensions in the tensor (5)